<a href="https://colab.research.google.com/github/bosbintang/python1/blob/main/Crop_optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pulp

In [ ]:
import pulp
from typing import Iterable
import numpy as np

In [ ]:
class Scenario:
  def __init__(self, start_week: int, end_week: int, productivity: float, budget: float):
    self._start_week = start_week
    self._end_week = end_week
    self._productivity = productivity
    self._budget = budget


class Vegetable:
  def __init__(self, name: str, spacing: float, scenarios: Iterable[Scenario] = []):
    _scenarios_input_validator(scenarios)
    self.name = name
    self.spacing = spacing
    self.scenarios = scenarios

  def add_scenario(self, scenario: Scenario):
    self.scenarios.append(scenario)

  def _build_scenario_matrix(self):
    """convert scenario into pulp variable"""
    scenario_matrix = list()
    scenario_vars = list()
    for i, scenario in enumerate(self.scenarios):
      scenario_name = "{}_S_{}".format(self.name, i+1)
      scenario_var = pulp.LpVariable(scenario_name, lowBound=0, cat="Integer")

      # put scenario_var in a 52week list, 0 if not within scenario week range
      # e.g. [tomato_s_1, tomato_s_1, .., 0 0 0 0 0 0 0]
      scenario_var_52week = [
        scenario_var
        if k+1 >= scenario._start_week and k+1<= scenario._end_week
        else 0
        for k in range(52)
      ]
      
      scenario_matrix.append(scenario_var_52week)
      scenario_vars.append(scenario_var)

    # insert this property to the object
    # convert scenario matrix as numpy array for easier matrix operation
    self.scenario_matrix = np.array(scenario_matrix)
    self.scenario_vars = np.array(scenario_vars)
    self.scenario_productivity = np.array([s._productivity for s in self.scenarios])
    self.scenario_budget = np.array([s._budget for s in self.scenarios])
    

def _scenarios_input_validator(scenarios):
  if isinstance(scenarios, Iterable):
    for item in scenarios:
      if not(isinstance(item, Scenario)):
        raise(Exception("Please input list of scenario as the arguments"))

In [ ]:
class UserInput:
  LAND_SIZE = 0
  BUDGET = 0
  __TARGET_YIELDS = []
  def add_target_yields(self, name, target):
    self.__TARGET_YIELDS.append((name, target))

  @property
  def TARGET_YIELDS(self):
    return dict(self.__TARGET_YIELDS)

class ModelBuilder:
  def __init__(self, vegetables: Iterable[Vegetable], user_input: UserInput, objective="max_output"):
    self.vegetables = vegetables
    self.problem = pulp.LpProblem('Problem', pulp.LpMaximize)
    self.user_input = user_input
    self.objective = objective
    self.status = "no status"

  def optimize(self):
    # build scenario matrix for each vegetables
    for veg in self.vegetables:
      veg._build_scenario_matrix()

    # TODO : add obj function
    self._add_objective_function()

    # define constraint
    self._add_constraints_area()
    self._add_constraints_budget()
    self._add_constraints_minimum_output()

    # solve the problems
    self.status = self.problem.solve()

  def _add_objective_function(self):
      if self.objective == "max_output":
        self._add_objective_max_output()
      else:
        raise(Exception("WRONG objective function"))

  def _add_objective_max_output(self):
    obj = pulp.lpSum([
      np.sum(veg.scenario_vars.dot(veg.scenario_productivity))
      for veg in self.vegetables
    ])
    self.problem += obj

  def _add_constraints_area(self):
    for k in range(52):
      # sum vertically for each week
      constr = pulp.lpSum([
        np.sum(vegs.scenario_matrix, axis=0)[k]*vegs.spacing
        for vegs in self.vegetables
      ]) <= self.user_input.LAND_SIZE
      self.problem += constr
  
  def _add_constraints_budget(self):
    self.problem += pulp.lpSum([
      np.sum(veg.scenario_vars.dot(veg.scenario_budget))
      for veg in self.vegetables
    ]) <= self.user_input.BUDGET
    
  def _add_constraints_minimum_output(self):
    for veg in self.vegetables:
      self.problem += (
          pulp.lpSum(veg.scenario_vars.dot(veg.scenario_productivity)) 
          >= self.user_input.TARGET_YIELDS.get(veg.name)
      )
    



-----------

# Example

In [ ]:
## Creating vegetable object (plant week start, plant week end, area used, budget used)
scenario_tomato = [
    Scenario(11,30,6,1000),
    Scenario(12,31,6,1000),
    Scenario(13,32,6,1000),
    Scenario(14,33,6,1000),
    Scenario(15,34,6,1000),
    Scenario(16,35,6,1000),
    Scenario(17,36,6,1000)
]

scenario_lettuce = [
    Scenario(9,21,3,1200),
    Scenario(10,22,3,1200),
    Scenario(11,23,3,1200),
    Scenario(12,24,3,1200),
    Scenario(13,25,3,1200),
    Scenario(32,43,3,1200),
    Scenario(33,44,3,1200),
    Scenario(34,45,3,1200),
    Scenario(35,46,3,1200),
    Scenario(36,47,3,1200)
]

scenario_carrot = [
    Scenario(8,24,4,1500),
    Scenario(9,25,4,1500),
    Scenario(10,26,4,1500),
    Scenario(11,27,4,1500),
    Scenario(12,28,4,1500),
    Scenario(13,29,4,1500),
    Scenario(27,44,4,1500),
    Scenario(28,45,4,1500),
    Scenario(29,46,4,1500),
    Scenario(30,47,4,1500)
]

scenario_eggplant = [
    Scenario(12,31,4,1000),
    Scenario(13,32,4,1000),
    Scenario(14,33,4,1000),
    Scenario(15,34,4,1000),
    Scenario(16,35,4,1000),
    Scenario(17,36,4,1000),
    Scenario(18,37,4,1000)
]

tomato = Vegetable('tomato', 10, scenario_tomato)
carrot = Vegetable('carrot', 10, scenario_carrot)
lettuce = Vegetable('lettuce', 10, scenario_lettuce)
eggplant = Vegetable('eggplant', 10, scenario_eggplant)

In [ ]:
# build user input

user_input = UserInput()
user_input.LAND_SIZE = 200
user_input.BUDGET = 100000
user_input.add_target_yields('tomato', 100)
user_input.add_target_yields('lettuce', 100)
user_input.add_target_yields('carrot', 100)
user_input.add_target_yields('eggplant', 100)

In [ ]:
# build optimization model

model = ModelBuilder([tomato, carrot, lettuce, eggplant], user_input)
model.optimize()

## TODO :
# model.result_summary()

In [ ]:
for veg in model.vegetables:
  for var, scenario in zip(veg.scenario_vars, veg.scenarios):
    if var.varValue > 0:
      print(
          "plant", 
          var.varValue, 
          "unit of", 
          veg.name, 
          "| Start at", 
          scenario._start_week, 
          "th week, end at", 
          scenario._end_week, 
          "th week."
      )

plant 30.0 unit of carrot | Start at 11 th week, end at 27 th week.
plant 25.0 unit of lettuce | Start at 33 th week, end at 44 th week.
plant 20.0 unit of lettuce | Start at 36 th week, end at 47 th week.
plant 25.0 unit of eggplant | Start at 13 th week, end at 32 th week.
plant 20.0 unit of eggplant | Start at 16 th week, end at 35 th week.


# Mathematical Model

## Objective Function 
Maximizing expected yields <br>
<center>$max(A + B + C - 0.5*(A - targetA + B - targetB + C - targetC)$</center>
(other option, maximizing land used, or minimizing deviation, or minimizing cost)
$S^{(i)} = \{s^{(i)}_1, s^{(i)}_2, ..., s^{(i)}_j, ..., s^{(i)}_{n_i}\}$ : is a matrix of scenario.... <br>
$s^{(i)}_j = \{s^{(i)}_{jk}\}_{k=1 ... 52}$<br>
$i$ : index of plant <br>
$j$ : index of scenario <br>
$k$ : index of week <br>
$N$ : number of vegetables <br>
$n_i$ : number of scenario of vegetable-i <br>
$A$ : total Area <br>
$a^{(i)}$ : allocated land for plant-i for each unit of $s_{jk}^{(i)}$<br>

So total area being used by plant i scenario j and week k is $s_{jk}^{(i)}*a^{(i)}$

## List of constraint
1.  For every week, land used should not exceed maximum land used <br>
for k=1...52, where k is index of week <br>
$\sum_{i=1}^{N} \sum_{j=1}^{n_i} s_{jk}^{(i)}*a^{(i)} \leq A $

2. Cost to plant should not exceed the budget
...

3. Expected yields should higher than minimum target
...

